## Initial Setup

In [ ]:
# Initial setup...
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table
import fitsio
from scipy import interpolate
import glob
import math
import os
import sys

import h5py
import bisect

import argparse

# v. 2.6.1 (which works with python 2.7) installed from 
#  https://imageio.readthedocs.io/en/stable/installation.html
import imageio

import matplotlib.pyplot as plt

%matplotlib inline

## User input

In [ ]:
# If debug=True, just plot/calculate synthetic mags for the 4.00days past merger KN 
debug=False

# Kasen (2017) models directory name:
#kasen_dirname='/data/des40.a/data/dtucker/DESGW_analysis/Kasen_Kilonova_Models_2017/kilonova_models'

kasen_dirname= '/Users/mwiesner/KDC/kasen'
kasen_filename='knova_d1_n10_m0.030_vk0.05_fd1.0_Xlan1e-3.0.h5'
kasen_pathname=os.path.join(kasen_dirname,kasen_filename)

# Redshift of kilonova...
z_kn = 0.0099

# CSV file containing bandpass:
# (Need to add LSST passbands...)
bandpassFile = '/Users/mwiesner/KDC/kasen/DES_STD_BANDPASSES_Y3A2_ugrizY.test.csv'

# Comma-separated list with no spaces of passbands to be used...
bandList = ['u','g','r','i','z','Y']

# bandpass color palette:
bandpassColors_dict = {'u':'#56b4e9',
                       'g':'#008060',
                       'r':'#ff4000',
                       'i':'#850000',
                       'z':'#6600cc',
                       'y':'#000000',
                       'Y':'#000000'
                      }
    
# Verbosity level of output to screen (0,1,2,...)
verbose = 0

# Output directory...
output_dirname = '/Users/mwiesner/KDC/kasen'
sed_dirname = '/Users/mwiesner/KDC/kasen/SEDs/'


## Synthetic Photometry Methods

Based on /data/des40.a/data/dtucker/Y6A1_abscal/calc_abmag.py, which itself made use of some variations of code developed by Keith Bechtol and Eli Rykoff for DES Y3 interstellar reddening corrections...


In [ ]:
# Create an argparse Namespace and run "calc_abmag(args)"...
#  (e.g., run_calc_abmag('u,g,r,i,z,Y', bandpassFile, tempFile, 'LAMBDA', 'Flam', 'Flam', verbose) )

def run_calc_abmag(bandList, bandpassFile, spectrumFile, colname_wave, colname_flam, flux_type, verbose):
    
    args = argparse.Namespace(bandList = bandList, 
                              bandpassFile = bandpassFile, 
                              spectrumFile = spectrumFile, 
                              colname_wave = colname_wave, 
                              colname_flam = colname_flam, 
                              flux_type = flux_type, 
                              verbose = verbose)
    
    if args.verbose > 0: print(args)

    status = calc_abmag(args)

    return status

    
    
#parser.add_argument('--bandList', help='comma-separated list with no spaces', default='g,r,i,z,Y')
#parser.add_argument('--bandpassFile', help='name of the input plan file', default='DES_STD_BANDPASSES_Y3A2_ugrizY.test.csv')
#parser.add_argument('--spectrumFile', help='name of the input plan file (can be CSV file or a synphot-style FITS file')
#parser.add_argument('--colname_wave', help='name of the wavelength column (in case of a CSV spectrumFile)', default='wave')
#parser.add_argument('--colname_flux', help='name of the flux column (in case of a CSV spectrumFile)', default='flux')
#parser.add_argument('--flux_type', help='type of flux (Flam [ergs/sec/cm**2/Angstrom] or Fnu [ergs/sec/cm**2/Hz])? ', default='Flam')
#parser.add_argument('--verbose', help='verbosity level of output to screen (0,1,2,...)', default=0, type=int)
    

In [ ]:
# Main method for calculating synthetic AB magnitudes...

def calc_abmag(args):

    #  Extract the bandList...
    bandList = args.bandList
    bandList = bandList.split(',')
    if args.verbose > 0:
        print('bandList: ', bandList)

    #  Extract the name of the bandpassFile...
    bandpassFile = args.bandpassFile
    if os.path.isfile(bandpassFile)==False:
        print("""bandpassFile %s does not exist...""" % (bandpassFile))
        print('Returning with error code 1 now...')
        return 1
    if args.verbose > 0:
        print('bandpassFile: ', bandpassFile)

    #  Extract the name of the spectrum file...
    spectrumFile = args.spectrumFile
    if os.path.isfile(spectrumFile)==False:
        print("""spectrumFile %s does not exist...""" % (spectrumFile))
        print('Returning with error code 1 now...')
        return 1
    if args.verbose > 0:
        print('spectrumFile: ', spectrumFile)

    # Try to determine spectrumFile type (FITS file or CSV file)...
    spectrumType = 'Unknown'
    try:
        hdulist = fits.open(spectrumFile)
        hdulist.close()
        spectrumType = 'FITS'
    except IOError:
        if args.verbose > 2:
            print("""spectrumFile %s is not a FITS file...""" % (spectrumFile))
        try:
            df_test = pd.read_csv(spectrumFile)
            spectrumType = 'CSV'
        except IOError:
            if args.verbose > 2:
                print("""spectrumFile %s is not a CSV file...""" % (spectrumFile))

    # Read in spectrumFile and create a SciPy interpolated function of the spectrum...
    if spectrumType == 'FITS':
        flux,wave_lo,wave_hi = getSpectrumSynphot(spectrumFile, fluxFactor=1.0)
    elif spectrumType == 'CSV':
        flux,wave_lo,wave_hi = getSpectrumCSV(spectrumFile, 
                                              colname_wave=args.colname_wave, colname_flam=args.colname_flam, 
                                              fluxFactor=1.0)
    else:
        print("""Spectrum file %s is of unknown type...""" % (spectrumFile))
        print('Returning with error code 1 now...')
        return 1

    # Read the bandpassFile into a Pandas DataFrame...
    df_resp = pd.read_csv(bandpassFile, comment='#')

    # Check to make sure the spectrumFile covers at least the same wavelength range
    #  as the bandpassFile...
    if ( (wave_lo > df_resp['LAMBDA'].min()) or (wave_hi < df_resp['LAMBDA'].max()) ):
        print("""WARNING:  %s does not cover the full wavelength range of %s""" % (spectrumFile, bandpassFile))
        print('Returning with error code 1 now...')
        return 1

    # Create wavelength_array and flux_array...
    delta_wavelength = 1.0 # angstroms
    wavelength_array = np.arange(wave_lo, wave_hi, delta_wavelength)
    flux_array = flux(wavelength_array)
    

    # If needed, convert flux from flam to fnu...
    if args.flux_type == 'Fnu':
        fnu_array = flux_array
    elif args.flux_type == 'Flam':
        c_kms = 299792.5        # speed of light in km/s
        c_ms = 1000.*c_kms      # speed of light in m/s
        c_as = (1.000e10)*c_ms  # speed of light in Angstroms/sec
        fnu_array = flux_array * wavelength_array * wavelength_array / c_as
    else:
        print("""Flux type %s is unknown...""" % (args.flux_type))
        print('Returning with error code 1 now...')
        return 1


    # Print out header...
    outputLine = ''
    for band in bandList:
        outputLine =  """%s,%s""" % (outputLine, band)
    print(outputLine[1:])

    outputLine = ''
    for band in bandList:

        response = interpolate.interp1d(df_resp['LAMBDA'], df_resp[band],
                                        bounds_error=False, fill_value=0.,
                                        kind='linear')
        response_array = response(wavelength_array)

        try:
            abmag = calc_abmag_value(wavelength_array, response_array, fnu_array)
        except Exception:
            abmag = -9999.

        outputLine =  """%s,%.4f""" % (outputLine, abmag)

    print(outputLine[1:])

    return 0

In [ ]:
# Calculate abmag using the wavelength version of the Fukugita et al. (1996) equation...

def calc_abmag_value(wavelength_array, response_array, fnu_array):

    # Calculate the abmag...
    numerator = np.sum(fnu_array * response_array / wavelength_array)
    denominator = np.sum(response_array / wavelength_array)
    abmag_value = -2.5*math.log10(numerator/denominator) - 48.60

    return abmag_value

In [ ]:
# Return a SciPy interpolation function of a Synphot-style FITS spectrum...
#  (Based on code from Keith Bechtol's synthesize_locus.py.)
# Unless otherwise noted, fluxes are assumed to be Flam and wavelengths
#  are assumed to be in Angstroms...

def getSpectrumSynphot(synphotFileName, fluxFactor=1.0):

    try:

        hdulist = fits.open(synphotFileName)
        t = Table.read(hdulist[1])
        hdulist.close()

    except IOError:

        print("""Could not read %s""" % synphotFileName)
        sys.exit(1)


    wave = t['WAVELENGTH'].data.tolist()
    wave_lo = min(wave)
    wave_hi = max(wave)
    t['FLUX'] = fluxFactor*t['FLUX']
    flam = t['FLUX'].data.tolist()
    flam = t['FLUX'].data.tolist()
    data = {'wavelength': wave, 'flux': flam}

    f = interpolate.interp1d(data['wavelength'], data['flux'],
                             bounds_error=True,
                             kind='linear')

    return f,wave_lo,wave_hi

In [ ]:
# Return a SciPy interpolation function of a CSV-style spectrum...
#  (Based on code from Keith Bechtol's synthesize_locus.py.)
# Unless otherwise noted, fluxes are assumed to be Flam and wavelengths
#  are assumed to be in Angstroms...

def getSpectrumCSV(csvFileName, colname_wave='wave', colname_flam='flux', fluxFactor=1.0):

    try:

        df = pd.read_csv(csvFileName)

    except IOError:

        print("""Could not read %s""" % csvFileName)
        sys.exit(1)


    columnNameList = df.columns.tolist()

    if colname_wave not in columnNameList:
        print("""Column %s not in %s""" % (colname_wave, csvFileName))
        sys.exit(1)

    if colname_flam not in columnNameList:
        print("""Column %s not in %s""" % (colname_wave, csvFileName))
        sys.exit(1)

    wave = df[colname_wave].tolist()
    wave_lo = min(wave)
    wave_hi = max(wave)
    df[colname_flam] = fluxFactor*df[colname_flam]
    flam = df[colname_flam].tolist()
    data = {'wavelength': wave, 'flux': flam}

    f = interpolate.interp1d(data['wavelength'], data['flux'],
                             bounds_error=True,
                             kind='linear')

    return f,wave_lo,wave_hi

## Other Useful Methods

In [ ]:
# Redshift to luminosity distance...
#  Default values of H0 and Omega0 are from Bennett et al. (2014)...
def zToDlum(z, H0=69.6, Om0=0.286):
    from astropy.cosmology import FlatLambdaCDM
    cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)
    # comoving distance...
    Dcom = cosmo.comoving_distance(z)
    # luminosity distance...
    Dlum = (1.+z)*Dcom
    return Dlum


# Mpc_to_cm...
def Mpc_to_cm(Dmpc):
    Dcm = Dmpc*1.00e6*3.086e+18
    return Dcm

## Read in bandpass file

In [ ]:
df_band = pd.read_csv(bandpassFile, comment='#')

## Read in and Plot Kilonova SED...

In [ ]:
c = 2.99e10   # speed of light in cm/sec...

# open model file
fin    = h5py.File(kasen_pathname,'r')

# frequency in Hz
nu    = np.array(fin['nu'],dtype='d')
# array of time in seconds
times = np.array(fin['time'])
# covert time to days
times = times/3600.0/24.0

# specific luminosity (ergs/s/Hz)
# this is a 2D array, Lnu[times][nu]
Lnu_all   = np.array(fin['Lnu'],dtype='d')

# loop over times [in days] after merger:
for t_kn in times:
    
    if t_kn < 0.00: continue
    
    if debug == True:  t_kn = 4.0

    print(t_kn)

    try:
        # index corresponding to t_kn
        it = bisect.bisect(times,t_kn)
        # spectrum at this epoch
        Lnu = Lnu_all[it,:]
    except:
        print("Index out of bounds?")
        continue


    # Convert to in Llam (ergs/s/Angstrom)...
    lam0  = c/nu*1e8          # rest-frame wavelength
    lam   = lam0*(1+z_kn)     # redshifted wavelength
    Llam = Lnu*nu**2.0/c/1e8  # Llam

    df_model = pd.DataFrame({'LAMBDA0':lam0, 'LAMBDA':lam, 'Llam':Llam})

    #print min(lam), max(lam)
    wavelength_array = np.arange(min(lam), max(lam), 1.0)

    spec_flux_model = interpolate.interp1d(df_model.LAMBDA, df_model.Llam,bounds_error=False, fill_value=0.,kind='linear')
    spec_flux_model_array = spec_flux_model(wavelength_array)

    df_model_new = pd.DataFrame({'LAMBDA':wavelength_array, 'Llam':spec_flux_model_array})

    #norm = df_model_new['Llam'].median()
    norm = df_model_new[( (df_model_new.LAMBDA > 3000.) & (df_model_new.LAMBDA < 12000.) )].Llam.max()

    df_model_new['normLlam'] = df_model_new['Llam']/norm

    #ax = df_model_new.plot('LAMBDA','normLlam', c='#000000', label='Model', fontsize=18)
    ax = df_model_new.plot('LAMBDA','normLlam', c='#888888', label='Model')
    
    
    outputFile1 = sed_dirname+'sed_'+"{:.2f}".format(t_kn)+'.spec'
    #outputFile1 = os.path.join(sed_dirname, outputFile1)
    df_model_new.to_csv(outputFile1,mode='w')

    for band in bandList:
        #print band
        df_band.plot('LAMBDA', band, c=bandpassColors_dict[band], ax=ax)

    title = """%s""" % (kasen_filename)
    plt.title(title, fontsize=16)
    plt.xlim([3000., 11000.])
    plt.ylim([0.,1.1])
    #ax.legend(loc='upper right', fontsize=14, framealpha=0.5)
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    ax.set_xlabel('wavelength (observed frame) [$\\AA$]',fontsize=16)
    ax.set_ylabel('Relative $F_{\lambda}$',fontsize=16)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 16)
    textstr = """$z$=%.3f\n$t$=%.2f days""" % (z_kn, t_kn)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
    ax.grid(False)
    
  #  outputFile = """KNspectrum.%s_z%.3f_a%05.2f.png""" % (kasen_filename, z_kn, t_kn)
  #  outputFile = os.path.join(output_dirname, outputFile)
    
    #plt.tight_layout()
   # plt.savefig(outputFile)

    if debug == True:  break



## Create Animated Gif
See https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python, as suggested by S. Allam.

In [174]:
# Define name of the animated GIF to be output...    
outputAnimGif = """KNspectrum.%s_z%.3f.gif""" % (kasen_filename, t_kn)
outputAnimGif = os.path.join(output_dirname, outputAnimGif)

# Identify input png files for the animated GIF...
filenameTemplate = """KNspectrum.%s_z%.3f_a??.??.png""" % (kasen_filename, z_kn)
filenameTemplate = os.path.join(output_dirname, filenameTemplate)
#print filenameTemplate
filenames = glob.glob(filenameTemplate)
#for filename in filenames:
#    print filename


# Generate animated GIF...
with imageio.get_writer(outputAnimGif, mode='I', duration=0.1) as writer:
    for filename in filenames:
        print(filename)
        image = imageio.imread(filename)
        writer.append_data(image)

print("""Animated GIF can be found here:  %s""" % (outputAnimGif))

Animated GIF can be found here:  /Users/mwiesner/KDC/kasen/KNspectrum.knova_d1_n10_m0.030_vk0.05_fd1.0_Xlan1e-3.0.h5_z10.950.gif


## Display Animated Gif
See https://github.com/ipython/ipython/issues/10045#issuecomment-608641627

In [82]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display

# Display GIF in Jupyter, CoLab, IPython
with open(outputAnimGif,'rb') as f:
    display.Image(data=f.read(), format='png')

## Write final spectrum model to CSV file (example):

In [83]:
#df_model_new.to_csv('test.csv', index=False)
#!head -50  test.csv

## Read in KN SED and Calculate KN Synthetic Magnitudes

In [84]:
c = 2.99e10   # speed of light in cm/sec...

# open model file
fin    = h5py.File(kasen_pathname,'r')

# frequency in Hz
nu    = np.array(fin['nu'],dtype='d')
# array of time in seconds
times = np.array(fin['time'])
# covert time to days
times = times/3600.0/24.0

# specific luminosity (ergs/s/Hz)
# this is a 2D array, Lnu[times][nu]
Lnu_all   = np.array(fin['Lnu'],dtype='d')

# loop over times [in days] after merger:
for t_kn in times:
    
    if t_kn < 0.00: continue

    if debug == True:  t_kn = 4.0

    print(t_kn,) 

    # index corresponding to t
    it = bisect.bisect(times,t_kn)
    
    # spectrum at this epoch
    Lnu = Lnu_all[it,:]

    # Convert to in Llam (ergs/s/Angstrom)...
    lam0  = c/nu*1e8          # rest-frame wavelength
    lam   = lam0*(1+z_kn)     # redshifted wavelength
    Llam = Lnu*nu**2.0/c/1e8  # Llam

    df_model = pd.DataFrame({'LAMBDA0':lam0, 'LAMBDA':lam, 'Llam':Llam})

    #print min(lam), max(lam)
    wavelength_array = np.arange(min(lam), max(lam), 1.0)

    spec_flux_model = interpolate.interp1d(df_model.LAMBDA, df_model.Llam,bounds_error=False, fill_value=0.,kind='linear')
    spec_flux_model_array = spec_flux_model(wavelength_array)

    df_model_new = pd.DataFrame({'LAMBDA':wavelength_array, 'Llam':spec_flux_model_array})

    # Distance to KN in cm...
    Dlum = zToDlum(z_kn)
    Dlum_cm = Mpc_to_cm(Dlum)

    # Calculate Flam [ergs/s/cm**2/Angstrom] from Llam [ergs/s/Angstrom] and Dlum_cm [cm]...
    df_model_new['Flam'] = df_model_new['Llam']/(Dlum_cm*Dlum_cm)

    # Write df_model_new to a temporary output file...
    tempFile = os.path.join(output_dirname,'temp_KNSpectrum.csv')
    df_model_new.to_csv(tempFile, index=False)

    run_calc_abmag('u,g,r,i,z,Y', bandpassFile, tempFile, 'LAMBDA', 'Flam', 'Flam', verbose)
    
    if debug == True:  break

0.05
u,g,r,i,z,Y
18.5166,18.2309,18.0471,18.2128,18.3667,18.3022


    Header size is not multiple of 2880: 4152158
There may be extra bytes after the last HDU or the file is corrupted. [astropy.io.fits.hdu.hdulist]


0

0.14999999
u,g,r,i,z,Y
17.8009,17.4383,17.2115,17.3857,17.5426,17.6123


    Header size is not multiple of 2880: 4607852
There may be extra bytes after the last HDU or the file is corrupted. [astropy.io.fits.hdu.hdulist]


...

24.85
u,g,r,i,z,Y
-9999.0000,27.6865,23.3361,21.2632,20.7904,21.4049


    Header size is not multiple of 2880: 5579200
There may be extra bytes after the last HDU or the file is corrupted. [astropy.io.fits.hdu.hdulist]


0

24.949999


IndexError: index 270 is out of bounds for axis 0 with size 270